In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from keras.models import Sequential, Model
from keras.layers import Convolution2D, MaxPooling2D, Conv3D, MaxPooling3D, ZeroPadding3D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Input
from keras.utils.np_utils import to_categorical
from sklearn.decomposition import PCA
from keras.optimizers import Adam, SGD, Adadelta, RMSprop, Nadam
import keras.callbacks as kcallbacks
from keras.regularizers import l2
import time
import collections
from sklearn import metrics, preprocessing

from Utils import zeroPadding, normalization, doPCA, modelStatsRecord, averageAccuracy, ssrn_SS_IN
print('Finished importing modules in *SSRN_IN.py* at ' + time.ctime(time.time()))

Using TensorFlow backend.


Finished importing modules in *SSRN_IN.py* at Thu May  2 16:37:41 2019


In [2]:
def indexToAssignment(index_, Row, Col, pad_length):
    new_assign = {}
    for counter, value in enumerate(index_):
        assign_0 = value // Col + pad_length
        assign_1 = value % Col + pad_length
        new_assign[counter] = [assign_0, assign_1]
    return new_assign
print('Finished *indexToAssignment* modules in *SSRN_IN.py* at ' + time.ctime(time.time()))

Finished *indexToAssignment* modules in *SSRN_IN.py* at Thu May  2 16:37:41 2019


In [3]:
def assignmentToIndex( assign_0, assign_1, Row, Col):
    new_index = assign_0 * Col + assign_1
    return new_index
print('Finished *assignmentToIndex* in *SSRN_IN.py* at ' + time.ctime(time.time()))

Finished *assignmentToIndex* in *SSRN_IN.py* at Thu May  2 16:37:41 2019


In [4]:
def selectNeighboringPatch(matrix, pos_row, pos_col, ex_len):
    selected_rows = matrix[range(pos_row-ex_len,pos_row+ex_len+1), :]
    selected_patch = selected_rows[:, range(pos_col-ex_len, pos_col+ex_len+1)]
    return selected_patch
print('Finished *selectNeighboringPatch* in *SSRN_IN.py* at ' + time.ctime(time.time()))

Finished *selectNeighboringPatch* in *SSRN_IN.py* at Thu May  2 16:37:41 2019


In [5]:
    def sampling(proptionVal, groundTruth):              
    #divide dataset into train and test datasets
        labels_loc = {}
        train = {}
        test = {}
        m = max(groundTruth)
        for i in range(m):
            indices = [j for j, x in enumerate(groundTruth.ravel().tolist()) if x == i + 1]
            np.random.shuffle(indices)
            labels_loc[i] = indices
            nb_val = int(proptionVal * len(indices))
            train[i] = indices[:-nb_val]
            test[i] = indices[-nb_val:]
    #    whole_indices = []
        train_indices = []
        test_indices = []
        for i in range(m):
    #        whole_indices += labels_loc[i]
            train_indices += train[i]
            test_indices += test[i]
        np.random.shuffle(train_indices)
        np.random.shuffle(test_indices)
        return train_indices, test_indices
    print('Finished *sampling* in *SSRN_IN.py* at ' + time.ctime(time.time()))

Finished *sampling* in *SSRN_IN.py* at Thu May  2 16:37:41 2019


In [6]:
def res4_model_ss():
    model_res4 = ssrn_SS_IN.ResnetBuilder.build_resnet_8((1, img_rows, img_cols, img_channels), nb_classes)

    RMS = RMSprop(lr=0.0003)
    # Let's train the model using RMSprop
    model_res4.compile(loss='categorical_crossentropy', optimizer=RMS, metrics=['accuracy'])

    return model_res4
print('Finished *res4modelss* in *SSRN_IN.py* at ' + time.ctime(time.time()))

Finished *res4modelss* in *SSRN_IN.py* at Thu May  2 16:37:41 2019


In [7]:
mat_data = sio.loadmat('C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/datasets/IN/Indian_pines_corrected.mat')
data_IN = mat_data['indian_pines_corrected']
mat_gt = sio.loadmat('C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/datasets/IN/Indian_pines_gt.mat')
gt_IN = mat_gt['indian_pines_gt']
print('Finished module in *SSRN_IN.py* at ' + time.ctime(time.time()))

Finished module in *SSRN_IN.py* at Thu May  2 16:37:41 2019


In [8]:
#new_gt_IN = set_zeros(gt_IN, [1,4,7,9,13,15,16])
new_gt_IN = gt_IN

batch_size = 16
nb_classes = 16
nb_epoch = 200  #400
img_rows, img_cols = 7, 7         #27, 27
patience = 200

INPUT_DIMENSION_CONV = 200
INPUT_DIMENSION = 200
TOTAL_SIZE = 10249
VAL_SIZE = 1025

TRAIN_SIZE = 2055
TEST_SIZE = TOTAL_SIZE - TRAIN_SIZE
VALIDATION_SPLIT = 0.8                      # 20% for trainnig and 80% for validation and testing
# TRAIN_NUM = 10
# TRAIN_SIZE = TRAIN_NUM * nb_classes
# TEST_SIZE = TOTAL_SIZE - TRAIN_SIZE
# VAL_SIZE = TRAIN_SIZE
img_channels = 200
PATCH_LENGTH = 3                #Patch_size (13*2+1)*(13*2+1)

data = data_IN.reshape(np.prod(data_IN.shape[:2]),np.prod(data_IN.shape[2:]))
gt = new_gt_IN.reshape(np.prod(new_gt_IN.shape[:2]),)

data = preprocessing.scale(data)

# scaler = preprocessing.MaxAbsScaler()
# data = scaler.fit_transform(data)


data_ = data.reshape(data_IN.shape[0], data_IN.shape[1],data_IN.shape[2])
whole_data = data_
padded_data = zeroPadding.zeroPadding_3D(whole_data, PATCH_LENGTH)

ITER = 1
CATEGORY = 16

train_data = np.zeros((TRAIN_SIZE, 2*PATCH_LENGTH + 1, 2*PATCH_LENGTH + 1, INPUT_DIMENSION_CONV))
test_data = np.zeros((TEST_SIZE, 2*PATCH_LENGTH + 1, 2*PATCH_LENGTH + 1, INPUT_DIMENSION_CONV))

KAPPA_RES_SS4 = []
OA_RES_SS4 = []
AA_RES_SS4 = []
TRAINING_TIME_RES_SS4 = []
TESTING_TIME_RES_SS4 = []
ELEMENT_ACC_RES_SS4 = np.zeros((ITER, CATEGORY))

#seeds = [1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229]

seeds = [1334]

for index_iter in range(ITER):
    print("# %d Iteration" % (index_iter + 1))

    # save the best validated model 
    
    best_weights_RES_path_ss4 = r'''C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/models/Indian_best_RES_3D_SS4_10_''' + str(
        index_iter + 1) + '.hdf5'

    np.random.seed(seeds[index_iter])
#    train_indices, test_indices = sampleFixNum.samplingFixedNum(TRAIN_NUM, gt)
    train_indices, test_indices = sampling(VALIDATION_SPLIT, gt)

    # TRAIN_SIZE = len(train_indices)
    # print (TRAIN_SIZE)
    #
    # TEST_SIZE = TOTAL_SIZE - TRAIN_SIZE - VAL_SIZE
    # print (TEST_SIZE)

    y_train = gt[train_indices] - 1
    y_train = to_categorical(np.asarray(y_train))

    y_test = gt[test_indices] - 1
    y_test = to_categorical(np.asarray(y_test))

    # print ("Validation data:")
    # collections.Counter(y_test_raw[-VAL_SIZE:])
    # print ("Testing data:")
    # collections.Counter(y_test_raw[:-VAL_SIZE])

    train_assign = indexToAssignment(train_indices, whole_data.shape[0], whole_data.shape[1], PATCH_LENGTH)
    for i in range(len(train_assign)):
        train_data[i] = selectNeighboringPatch(padded_data, train_assign[i][0], train_assign[i][1], PATCH_LENGTH)

    test_assign = indexToAssignment(test_indices, whole_data.shape[0], whole_data.shape[1], PATCH_LENGTH)
    for i in range(len(test_assign)):
        test_data[i] = selectNeighboringPatch(padded_data, test_assign[i][0], test_assign[i][1], PATCH_LENGTH)

    x_train = train_data.reshape(train_data.shape[0], train_data.shape[1], train_data.shape[2], INPUT_DIMENSION_CONV)
    x_test_all = test_data.reshape(test_data.shape[0], test_data.shape[1], test_data.shape[2], INPUT_DIMENSION_CONV)

    x_val = x_test_all[-VAL_SIZE:]
    y_val = y_test[-VAL_SIZE:]

    x_test = x_test_all[:-VAL_SIZE]
    y_test = y_test[:-VAL_SIZE]

    # SS Residual Network 4 with BN
    model_res4_SS_BN = res4_model_ss()

    earlyStopping6 = kcallbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=1, mode='auto')
    saveBestModel6 = kcallbacks.ModelCheckpoint(best_weights_RES_path_ss4, monitor='val_loss', verbose=1,
                                                save_best_only=True,
                                                mode='auto')

    tic6 = time.clock()
    print(x_train.shape, x_test.shape)
    history_res4_SS_BN = model_res4_SS_BN.fit(
        x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], x_train.shape[3], 1), y_train,
        validation_data=(x_val.reshape(x_val.shape[0], x_val.shape[1], x_val.shape[2], x_val.shape[3], 1), y_val),
        batch_size=batch_size,
        nb_epoch=nb_epoch, shuffle=True, callbacks=[earlyStopping6, saveBestModel6])
    toc6 = time.clock()

    tic7 = time.clock()
    loss_and_metrics_res4_SS_BN = model_res4_SS_BN.evaluate(
        x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], x_test.shape[3], 1), y_test,
        batch_size=batch_size)
    toc7 = time.clock()

    print('3D RES_SS4 without BN Training Time: ', toc6 - tic6)
    print('3D RES_SS4 without BN Test time:', toc7 - tic7)

    print('3D RES_SS4 without BN Test score:', loss_and_metrics_res4_SS_BN[0])
    print('3D RES_SS4 without BN Test accuracy:', loss_and_metrics_res4_SS_BN[1])

    print(history_res4_SS_BN.history.keys())

    pred_test_res4 = model_res4_SS_BN.predict(
        x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], x_test.shape[3], 1)).argmax(axis=1)
    collections.Counter(pred_test_res4)
    gt_test = gt[test_indices] - 1
    overall_acc_res4 = metrics.accuracy_score(pred_test_res4, gt_test[:-VAL_SIZE])
    confusion_matrix_res4 = metrics.confusion_matrix(pred_test_res4, gt_test[:-VAL_SIZE])
    each_acc_res4, average_acc_res4 = averageAccuracy.AA_andEachClassAccuracy(confusion_matrix_res4)
    kappa = metrics.cohen_kappa_score(pred_test_res4, gt_test[:-VAL_SIZE])
    KAPPA_RES_SS4.append(kappa)
    OA_RES_SS4.append(overall_acc_res4)
    AA_RES_SS4.append(average_acc_res4)
    TRAINING_TIME_RES_SS4.append(toc6 - tic6)
    TESTING_TIME_RES_SS4.append(toc7 - tic7)
    ELEMENT_ACC_RES_SS4[index_iter, :] = each_acc_res4

    print("3D RESNET_SS4 without BN training finished.")
    print("# %d Iteration" % (index_iter + 1))

modelStatsRecord.outputStats(KAPPA_RES_SS4, OA_RES_SS4, AA_RES_SS4, ELEMENT_ACC_RES_SS4,
                             TRAINING_TIME_RES_SS4, TESTING_TIME_RES_SS4,
                             history_res4_SS_BN, loss_and_metrics_res4_SS_BN, CATEGORY,
                             r'''C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/records/IN_train_SS_10.txt''',
                             r'''C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/records/IN_train_SS_element_10.txt''')

print('Finished running *SSRN_IN.py* at ' + time.ctime(time.time()))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype uint16 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


# 1 Iteration
input shape: 200
Instructions for updating:
Colocations handled automatically by placer.
input shape: (None, 7, 7, 97, 24)
conv_spc_result shape: (None, 7, 7, 1, 128)
conv1 shape: (None, 5, 5, 1, 24)
input shape: (None, 5, 5, 1, 24)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
(2055, 7, 7, 200) (7169, 7, 7, 200)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:109: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Instructions for updating:
Use tf.cast instead.
Train on 2055 samples, validate on 1025 samples
Epoch 1/200
2055/2055 [==============================] - 53s 26ms/step - loss: 2.3399 - acc: 0.2526 - val_loss: 1.5680 - val_acc: 0.6741

Epoch 00001: val_loss improved from inf to 1.56798, saving model to C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/models/Indian_best_RES_3D_SS4_10_1.hdf5
Epoch 2/200
2055/2055 [==============================] - 45s 22ms/step - loss: 1.8600 - acc: 0.4336 - val_loss: 1.2234 - val_acc: 0.7707

Epoch 00002: val_loss improved from 1.56798 to 1.22342, saving model to C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/models/Indian_best_RES_3D_SS4_10_1.hdf5
Epoch 3/200
2055/2055 [==============================] - 45s 22ms/step - loss: 1.5584 - acc: 0.5596 - val_loss: 1.0900 - val_acc: 0.8039

Epoch 00003: val_loss improved from 1.22342 to 1.08996, saving model to C:/Use

2055/2055 [==============================] - 44s 21ms/step - loss: 0.3648 - acc: 0.9002 - val_loss: 0.1113 - val_acc: 0.9844

Epoch 00030: val_loss improved from 0.11478 to 0.11125, saving model to C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/models/Indian_best_RES_3D_SS4_10_1.hdf5
Epoch 31/200
2055/2055 [==============================] - 45s 22ms/step - loss: 0.3761 - acc: 0.9002 - val_loss: 0.2001 - val_acc: 0.9551

Epoch 00031: val_loss did not improve from 0.11125
Epoch 32/200
2055/2055 [==============================] - 47s 23ms/step - loss: 0.3248 - acc: 0.9158 - val_loss: 0.1046 - val_acc: 0.9863

Epoch 00032: val_loss improved from 0.11125 to 0.10456, saving model to C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/models/Indian_best_RES_3D_SS4_10_1.hdf5
Epoch 33/200
2055/2055 [==============================] - 45s 22ms/step - loss: 0.3540 - acc: 0.8998 - val_loss: 0.1034 - val_acc

...

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:116: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:118: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


7169/7169 [==============================] - 49s 7ms/step
3D RES_SS4 without BN Training Time:  9301.6052992
3D RES_SS4 without BN Test time: 49.05255080000097
3D RES_SS4 without BN Test score: 0.07826613940995912
3D RES_SS4 without BN Test accuracy: 0.9931650160412889
dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:122: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


3D RESNET_SS4 without BN training finished.
# 1 Iteration
Test score: 0.07826613940995912
Test accuracy: 0.9931650160412889
dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])
Finished running *SSRN_IN.py* at Thu May  2 19:14:34 2019


C:\Users\josep\Desktop\RS\residualnet_tensorflow_keras\residualnet_tensorflow_keras\SSRN-master\Utils\averageAccuracy.py:8: RuntimeWarning: invalid value encountered in true_divide
  each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
